In [1]:
import os 
import matplotlib
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import random
import cutde.fullspace as FS
import plotly
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
# from scipy import sparse
from scipy.linalg import svd, lstsq
from scipy.integrate import cumulative_trapezoid
from scipy.signal import resample, detrend,medfilt
from scipy.stats import bootstrap
from operator import itemgetter
import matplotlib.dates as mdates
from datetime import datetime
from more_itertools import sliced
from numpy.linalg import norm
from numpy.linalg import solve
# import scipy.interpolate as sp
import cmasher as cmr
import pickle
from dts_funcs import * 

In [2]:
# Read in DAS data
def stack_DAS(das_dat):
    n = das_dat.shape[1]
    m = n // 2
    a1 = das_dat[:,1:m+1]
    a2 = np.fliplr(das_dat[:,m+1:])
    return  np.divide(np.add(a1,a2),2)

df_fullPDT = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellPDT/maystim22_26_combined_full')
df_fullPDB = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellPDB/maystim22_26_combined_full')
df_fullPST = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellPST/maystim22_26_combined_full')
df_fullPSB = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellPSB/maystim22_26_combined_full')
df_fullOT = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellOT/maystim22_26_combined_full')
df_fullOB = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellOB/maystim22_26_combined_full')

df_fullPDT = detrend(df_fullPDT,axis=0,type='linear')
df_fullPDB = detrend(df_fullPDB,axis=0,type='linear')
df_fullPST = detrend(df_fullPST,axis=0,type='linear')
df_fullPSB = detrend(df_fullPSB,axis=0,type='linear')
df_fullOT = detrend(df_fullOT,axis=0,type='linear')
df_fullOB = detrend(df_fullOB,axis=0,type='linear')

# df_fullPDT = stack_DAS(df_fullPDT)
# df_fullPDB = stack_DAS(df_fullPDB)
# df_fullPST = stack_DAS(df_fullPST)
# df_fullPSB = stack_DAS(df_fullPSB)
# df_fullOT = stack_DAS(df_fullOT)
# df_fullOB = stack_DAS(df_fullOB)


wn     = ['PDT','PDB','PST','PSB','OT','OB']
nfile_list = sorted(os.walk('/data1/parker/EGS_iDAS'))
nfile_list = nfile_list[1:]
#file_list = file_list[1:]
nfile_list = [group[2] for group in nfile_list]
nfile_list = [item for sublist in nfile_list for item in sublist]
# [file_list.append(f) for f in nfile_list]
fd = [name.split("_") for name in nfile_list]
fl = [fd[file][2].split(".") for file in range(len(fd))]
fl = [el[0] for el in fl]
DASdates = [datetime.strptime(d,'%y%m%d%H%M%S') for d in sorted(fl)]
# these are files that get skipped during the low pass filtering process and so the dates need to be removed 
ind2rem = [0, 90, 91, 257, 258, 1571, 1572, 3082, 3083, 5085, 5086, 5599, 5600, 5961, 5962, 7623, 7624, 8841, 8842, 9562]
# remove in reverse so that the indices remain in the correct order for removal
for index in sorted(ind2rem,reverse=True):
    del DASdates[index]

directory = sorted(os.walk('/home/spri902/mayCASSM'))
CASSMdates = [datetime.strptime(d,'%Y%m%d%H%M%S') for d in sorted(directory[0][1])]
chansT=np.linspace(0,df_fullPDT.shape[1] - 1,df_fullPDT.shape[1]).astype(int)
chansB=np.linspace(0,df_fullPDB.shape[1] - 1,df_fullPDB.shape[1]).astype(int)
# stimbeg = [96, 224, 352, 472, 507]
# stimfin = [106, 232, 368, 475, 512]
# stimfin = [106, 232, 368, 475, 511]
# stimbegLines = itemgetter(*stimbeg)(CASSMdates)
# stimfinLines = itemgetter(*stimfin)(CASSMdates)
stimbeg = [0, 2448, 5737, 7749, 8405]
# stimfin = [106, 232, 368, 475, 512]
stimfin = [29, 2578, 5813, 7807, 8469]
stimbegLines = itemgetter(*stimbeg)(DASdates)
stimfinLines = itemgetter(*stimfin)(DASdates)

dasdnums = mdates.date2num(DASdates)
cassmdnums = mdates.date2num(CASSMdates)
dasd = pd.date_range(start=DASdates[0],end=DASdates[-1],periods=len(cassmdnums[:-2]))

In [3]:
# scaling equation provided by Silixa (tdmsvalues/2048)*116*(fs/gaugeLength)

# set up matrices to scale the data by their different Fs values
fsvecT = np.ones_like(df_fullPDT)
fsvecB = np.ones_like(df_fullPDB)
fsvecOT = np.ones_like(df_fullOT)
fsvecOB = np.ones_like(df_fullOB)
fsvecPST = np.ones_like(df_fullPST)
fsvecPSB = np.ones_like(df_fullPSB)
# These are the indices where the sampling rate changes between 1kHz and 10kHz
lst = [0,88,10000,89,1565,1000,1566,5075,10000,5076,5587,1000,5588,7607,10000,7608,9543,1000]
lst = np.array(list(sliced((lst),3))).reshape(6,3)
# fill the matrices with the correct sampling freqs
for i in range(len(lst)):
    fsvecT[lst[i][0]:lst[i][1]+1,:] = fsvecT[lst[i][0]:lst[i][1]+1,:]*lst[i][2]
    fsvecB[lst[i][0]:lst[i][1]+1,:] = fsvecB[lst[i][0]:lst[i][1]+1,:]*lst[i][2]
    fsvecOT[lst[i][0]:lst[i][1]+1,:] = fsvecOT[lst[i][0]:lst[i][1]+1,:]*lst[i][2]
    fsvecOB[lst[i][0]:lst[i][1]+1,:] = fsvecOB[lst[i][0]:lst[i][1]+1,:]*lst[i][2]
    fsvecPST[lst[i][0]:lst[i][1]+1,:] = fsvecPST[lst[i][0]:lst[i][1]+1,:]*lst[i][2]
    fsvecPSB[lst[i][0]:lst[i][1]+1,:] = fsvecPSB[lst[i][0]:lst[i][1]+1,:]*lst[i][2]
# divide by gauge length
fsvecT /= 10.
fsvecB /= 10.
fsvecOT /= 10.
fsvecOB /= 10.
fsvecPST /= 10.
fsvecPSB /= 10.
# multiply the data by the scaling matrices and then scale by the Silixa scaling value
scaledT = np.multiply(fsvecT,df_fullPDT)*(116/2048)
scaledB = np.multiply(fsvecB,df_fullPDB)*(116/2048)
scaledOT = np.multiply(fsvecOT,df_fullOT)*(116/2048)
scaledOB = np.multiply(fsvecOB,df_fullOB)*(116/2048)
scaledPST = np.multiply(fsvecPST,df_fullPST)*(116/2048)
scaledPSB = np.multiply(fsvecPSB,df_fullPSB)*(116/2048)

# Integrate strain rates to strain
df_fullPDT = scaledT.copy()
df_fullPDB = scaledB.copy()
df_fullPST = scaledPST.copy()
df_fullPSB = scaledPSB.copy()
df_fullOT = scaledOT.copy()
df_fullOB = scaledOB.copy()
# Integrate strain rates to strain
df_strainPDT = cumulative_trapezoid(df_fullPDT,axis=0,initial=0)
df_strainPDB = cumulative_trapezoid(df_fullPDB,axis=0,initial=0)
df_strainPST = cumulative_trapezoid(df_fullPST,axis=0,initial=0)
df_strainPSB = cumulative_trapezoid(df_fullPSB,axis=0,initial=0)
df_strainOT = cumulative_trapezoid(df_fullOT,axis=0,initial=0)
df_strainOB = cumulative_trapezoid(df_fullOB,axis=0,initial=0)
# create channel spacing vectors
chansOB=np.linspace(0,df_strainOB.shape[1] - 1,df_strainOB.shape[1]).astype(int)
chansOT=np.linspace(0,df_strainOT.shape[1] - 1,df_strainOT.shape[1]).astype(int)
chansPSB=np.linspace(0,df_strainPSB.shape[1] - 1,df_strainPSB.shape[1]).astype(int)
chansPST=np.linspace(0,df_strainPST.shape[1] - 1,df_strainPST.shape[1]).astype(int)
chansPDB=np.linspace(0,df_strainPDB.shape[1] - 1,df_strainPDB.shape[1]).astype(int)
chansPDT=np.linspace(0,df_strainPDT.shape[1] - 1,df_strainPDT.shape[1]).astype(int)

In [4]:
# Read in well geometry
wellcols=np.arange(12)
WellGeom=pd.read_excel('/home/spri902/Collab_metadata/Well_Points.xlsx',header=0,usecols=wellcols)
WellGeom.columns = WellGeom.columns.str.replace(' ','')
# convert from feet to meters
WellGeom.x = WellGeom.x/3.28084
WellGeom.y = WellGeom.y/3.28084
WellGeom.z = WellGeom.z/3.28084
# pull out the monitoring wells
mwells=[]
wellList = ['E1-OB','E1-OT','E1-PDT','E1-PDB','E1-PST','E1-PSB']
for i in wellList:
    tmpwell = WellGeom[WellGeom["HoleID"]== i]
    tmpwell = tmpwell.iloc[:,0:6]
    mwells.append(tmpwell)
mwells = pd.concat(mwells)
# pull out the injector and production wells
swells=[]
wellList = ['E1-I','E1-P']
for i in wellList:
    tmpwell = WellGeom[WellGeom["HoleID"]== i]
    tmpwell = tmpwell.iloc[:,0:6]
    swells.append(tmpwell)
swells = pd.concat(swells)

# take every 10th point (1 meter spacing along well)
PDBpnts = mwells[mwells['HoleID'] == 'E1-PDB' ].iloc[0::10,:]
PDBpnts = PDBpnts[1:df_fullPDB.shape[1]]
PDTpnts = mwells[mwells['HoleID'] == 'E1-PDT' ].iloc[0::10,:]
PDTpnts = PDTpnts[1:df_fullPDT.shape[1]]

PSBpnts = mwells[mwells['HoleID'] == 'E1-PSB' ].iloc[0::10,:]
PSBpnts = PSBpnts[1:df_fullPSB.shape[1]]
PSTpnts = mwells[mwells['HoleID'] == 'E1-PST' ].iloc[0::10,:]
PSTpnts = PSTpnts[1:df_fullPST.shape[1]]

OBpnts = mwells[mwells['HoleID'] == 'E1-OB' ].iloc[0::10,:]
OBpnts = OBpnts[1:df_fullOB.shape[1]]
OTpnts = mwells[mwells['HoleID'] == 'E1-OT' ].iloc[0::10,:]
OTpnts = OTpnts[1:df_fullOT.shape[1]]


In [5]:
# well coordinates

pdtobsx = np.array(PDTpnts.x) # well PDT
pdbobsx = np.array(PDBpnts.x) # well PDB
pstobsx = np.array(PSTpnts.x) # well PST
psbobsx = np.array(PSBpnts.x) # well PSB
otobsx = np.array(OTpnts.x) # well OT
obobsx = np.array(OBpnts.x) # well OB

pdtobsy = np.array(PDTpnts.y)
pdbobsy = np.array(PDBpnts.y)
pstobsy = np.array(PSTpnts.y)
psbobsy = np.array(PSBpnts.y)
otobsy = np.array(OTpnts.y)
obobsy = np.array(OBpnts.y)

pdtobsz = np.array(PDTpnts.z)
pdbobsz = np.array(PDBpnts.z)
pstobsz = np.array(PSTpnts.z)
psbobsz = np.array(PSBpnts.z)
otobsz = np.array(OTpnts.z)
obobsz = np.array(OBpnts.z)

pdtpts = np.array([pdtobsx, pdtobsy,  pdtobsz]).reshape((3, -1)).T.copy()
pdbpts = np.array([pdbobsx, pdbobsy,  pdbobsz]).reshape((3, -1)).T.copy()
pstpts = np.array([pstobsx, pstobsy,  pstobsz]).reshape((3, -1)).T.copy()
psbpts = np.array([psbobsx, psbobsy,  psbobsz]).reshape((3, -1)).T.copy()
otpts = np.array([otobsx, otobsy,  otobsz]).reshape((3, -1)).T.copy()
obpts = np.array([obobsx, obobsy,  obobsz]).reshape((3, -1)).T.copy()

allpts = np.concatenate((pdtpts,pdbpts,pstpts,psbpts,otpts,obpts))

In [6]:
# Bring in MEQ catalog
meq = pd.read_csv('/home/spri902/EGS_Collab/4850/MEQ/MEQcat.csv',header=0,parse_dates=[11],infer_datetime_format=True)
meq.columns = meq.columns.str.replace(' ','')
dates = mdates.date2num(meq['time'])
meq['datenums'] = dates
meq = meq[(meq['time'] > pd.to_datetime('2018-05-24T21:00')) & (meq['time'] < pd.to_datetime('2018-05-25T23'))]
# meq = meq[(meq['time'] > pd.to_datetime('2018-05-25T20:30')) & (meq['time'] < pd.to_datetime('2018-05-25T21:35'))]

In [ ]:
# Bring in Injection data
os.chdir('/home/spri902/EGS_Collab/4850/stimflow/')
injFiles = sorted(os.listdir('/home/spri902/EGS_Collab/4850/stimflow/'))
injDat = pd.concat((pd.read_csv(f,header=1,usecols=[0,2,4,24],\
                           parse_dates = [0],infer_datetime_format=True) \
                    for f in injFiles if f.endswith('.csv')),axis=0)
injDat.rename(columns={'hh:mm:ss':'date','LPM':'QLPM','LPM.1':'TLPM','psig.9':'psig'},inplace=True)
injDat.reset_index(drop = True,inplace = True)
injDat.set_index('date',inplace=True)
injDat = injDat.iloc[14650:]
injDat = injDat[~injDat.index.duplicated(keep='first')]

cumQvol = np.cumsum(injDat.QLPM)
cumTvol = np.cumsum(injDat.TLPM)
flwRate = injDat.QLPM + injDat.TLPM

In [ ]:
# Read in DTS data and calculate temperature imposed strain
def calc_dts_strain(dts_list,well_list):


    dts_dat = {}
    for well in well_list:
        time = []
        deps = dts_list[0][well]['depth']
        dat =  []
        ref =  dts_list[0][well]['data'][:,0]
        for i in range(len(dts_list)):
            time.append(dts_list[i][well]['times'])
            # dat.append(dts_list[i][well]['data'])
            dat = np.concatenate([dts_list[x][well]['data']-ref[:,None] for x in range(len(dts_list))],axis=1)
            # dat = np.concatenate([dts_list[x][well]['data'] for x in range(len(dts_list))],axis=1)
        
        time = np.concatenate([t.flatten() for t in time])
    
        dts_dat[well] = {'data': dat, 'depth': deps,
                            'times': time}
    return dts_dat

# Create a list of dict DTS data from each pickle file
# Each list is len(dts_list) = number of pkl files
#  and Plot processed DTS images  
#
dts_list=[]
for path,subdir,files in sorted(os.walk('/home/spri902/EGS_Collab/4850/DTS/processed/data/Exp1')):
    for file in sorted(files):
        with open(os.path.join(path,f'{file}'),'rb') as f:
            well_dict = pickle.load(f)
            dts_list.append(well_dict)
for i in range(len(wells_4850)):    
    plot_DTS(dts_list,well=wells_4850[i])


# 
dts_strain = calc_dts_strain(dts_list,wn)
dts_times = dts_strain['OT']['times']

In [ ]:
# Pick a DTS record time as close as possible to a DAS record time
dts_t0 = 423
# dts_t0 = 523 # index for date time in DTS data
# dts_t0 = 557
print(dts_times[dts_t0])
# sum every 4 channels in DTS and take average to match DAS channel spacing
otdts = np.add.reduceat(dts_strain['OT']['data'][:,dts_t0],np.arange(0,len(dts_strain['OT']['data'][:,dts_t0]),4)) / 4
obdts = np.add.reduceat(dts_strain['OB']['data'][:,dts_t0],np.arange(0,len(dts_strain['OB']['data'][:,dts_t0]),4)) / 4
pstdts = np.add.reduceat(dts_strain['PST']['data'][:,dts_t0],np.arange(0,len(dts_strain['PST']['data'][:,dts_t0]),4)) / 4
psbdts = np.add.reduceat(dts_strain['PSB']['data'][:,dts_t0],np.arange(0,len(dts_strain['PSB']['data'][:,dts_t0]),4)) / 4
pdtdts = np.add.reduceat(dts_strain['PDT']['data'][:,dts_t0],np.arange(0,len(dts_strain['PDT']['data'][:,dts_t0]),4)) / 4
pdbdts = np.add.reduceat(dts_strain['PDB']['data'][:,dts_t0],np.arange(0,len(dts_strain['PDB']['data'][:,dts_t0]),4)) / 4


In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

def four_yaxes(ax,dastimes,chans,dasdat,flwdat,injdat,meqdat,colmap,flwcolor,prescolor,meqcol,wellnm,flag):
    ax1 = ax.twinx()
    ax2 = [ax,ax.twinx()]
    ax3 = ax.twinx()
    ax1.spines["right"].set_position(("axes",1.1))
    ax3.spines["right"].set_position(("axes",1.25))
    make_patch_spines_invisible(ax1)
    ax1.spines["right"].set_visible(True)
    make_patch_spines_invisible(ax3)
    ax3.spines["right"].set_visible(True)
    vm=np.nanpercentile(dasdat[7700:,:],99)
    img = ax.pcolormesh(dastimes,chans,dasdat.T,cmap=colmap,vmin=-vm,vmax=vm)
    ax.xaxis_date()
    ax.xaxis.set_tick_params(which='both',labelbottom=True)

    ax2[0].set_ylabel('Channel Number')
    im2 = ax2[1].plot(flwdat,'.',color=flwcolor,markersize=2,label='Flow Rate')
    ax2[1].set_ylabel('Flow Rate (LPM)')
    im3 = ax3.plot(injdat,'.',color=prescolor,markersize=2,label='Inj. Pressure')
    ax3.set_ylabel('Inj. Pressure (psig)')
    # im2[0].yaxis.label.set_color(ax2[1].get_color())
    ax3.yaxis.label.set_color(im3[0].get_color())
    ax2[1].yaxis.label.set_color(im2[0].get_color())
    im1 = ax1.plot(meqdat['datenums'],meqdat['D_OT_P'],'.',color=meqcol,label='MEQs')
    ax1.set_ylabel('MEQs distance from OTP (m)')
    ax1.yaxis.label.set_color(im1[0].get_color())
    ax1.plot(meqdat['datenums'],np.zeros_like(meqdat['datenums']),'-m')
    if flag:
        plt.colorbar(img,orientation="horizontal",label='Nanostrain Rate')
        plt.title(f'{wellnm} Strain Rate')
    else:
        plt.colorbar(img,orientation="horizontal",label='Nanostrain')
        plt.title(f'{wellnm} Strain')
    return img, im2, im3, im1
        


In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,2,figsize=(12,8),sharex=True,sharey=True)
fig.subplots_adjust(right=0.75)
a1,a1a,a1b,a1c = four_yaxes(ax1[0],DASdates,chansOT,df_fullOT,flwRate,injDat.loc[:,("psig")],meq,'seismic','cyan','green','orange',wn[4],1)
a2,a2a,a2b,a2c = four_yaxes(ax1[1],DASdates,chansOB,df_fullOB,flwRate,injDat.loc[:,("psig")],meq,'seismic','cyan','green','orange',wn[5],1)
a3,a3a,a3b,a3c = four_yaxes(ax2[0],DASdates,chansPDT,df_fullPDT,flwRate,injDat.loc[:,("psig")],meq,'seismic','cyan','green','orange',wn[0],1)
a4,a4a,a4b,a4c = four_yaxes(ax2[1],DASdates,chansPDB,df_fullPDB,flwRate,injDat.loc[:,("psig")],meq,'seismic','cyan','green','orange',wn[1],1)
a5,a5a,a5b,a5c = four_yaxes(ax3[0],DASdates,chansPST,df_fullPST,flwRate,injDat.loc[:,("psig")],meq,'seismic','cyan','green','orange',wn[2],1)
a6,a6a,a6b,a6c = four_yaxes(ax3[1],DASdates,chansPSB,df_fullPSB,flwRate,injDat.loc[:,("psig")],meq,'seismic','cyan','green','orange',wn[3],1)

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,2,figsize=(12,8),sharex=True,sharey=True)
fig.subplots_adjust(right=0.75)
a1,a1a,a1b,a1c = four_yaxes(ax1[0],DASdates,chansOT,df_strainOT,flwRate,injDat.loc[:,("psig")],meq,'seismic','black','green','orange',wn[4],0)
a2,a2a,a2b,a2c = four_yaxes(ax1[1],DASdates,chansOB,df_strainOB,flwRate,injDat.loc[:,("psig")],meq,'seismic','black','green','orange',wn[5],0)
a3,a3a,a3b,a3c = four_yaxes(ax2[0],DASdates,chansPDT,df_strainPDT,flwRate,injDat.loc[:,("psig")],meq,'seismic','black','green','orange',wn[0],0)
a4,a4a,a4b,a4c = four_yaxes(ax2[1],DASdates,chansPDB,df_strainPDB,flwRate,injDat.loc[:,("psig")],meq,'seismic','black','green','orange',wn[1],0)
a5,a5a,a5b,a5c = four_yaxes(ax3[0],DASdates,chansPST,df_strainPST,flwRate,injDat.loc[:,("psig")],meq,'seismic','black','green','orange',wn[2],0)
a6,a6a,a6b,a6c = four_yaxes(ax3[1],DASdates,chansPSB,df_strainPSB,flwRate,injDat.loc[:,("psig")],meq,'seismic','black','green','orange',wn[3],0)

plt.tight_layout()
plt.show()

In [ ]:
# switch dir to where the fracture points are
os.chdir('/home/spri902/EGS_Collab/4850/fractures/')
# Read in xyz points for OTP-connector and hydrofracs
with open('frac_plane_points.npy','rb') as f:
    x = np.load(f)

# Pull out OTP-connector points
otp = x[0:80,:]
westN = x[80:160,:]
# westN = interparc(80,westN[:,0],westN[:,1],westN[:,2]) 
westS = x[160:240,:]
westD = x[240:320,:]
eastN = x[320:400,:]
eastS = x[400:,:]
# otp = np.array(itemgetter(19,39,59)(x))
otpctr = otp.mean(axis=0)
wnctr = westN.mean(axis=0)
wsctr = westS.mean(axis=0)
wdctr = westD.mean(axis=0)
enctr = eastN.mean(axis=0)
esctr = eastS.mean(axis=0)

# fit the OT-P fracture plane with 4 of the 80 points
otpseedPnts = np.array([x[0,0],x[0,1],x[0,2],
x[19,0],x[19,1],x[19,2],
x[39,0],x[39,1],x[39,2],
x[59,0],x[59,1],x[59,2]]).reshape(4,3)
wnseedPnts = np.array([x[80,0],x[80,1],x[80,2],
x[99,0],x[99,1],x[99,2],
x[106,0],x[106,1],x[106,2],
x[131,0],x[131,1],x[131,2],x[141,0],x[141,1],x[141,2]]).reshape(5,3)
wsseedPnts = np.array([x[167,0],x[167,1],x[167,2],
x[187,0],x[187,1],x[187,2],
x[197,0],x[197,1],x[197,2],
x[215,0],x[215,1],x[215,2],x[225,0],x[225,1],x[225,2]]).reshape(5,3)
wdseedPnts = np.array([x[240,0],x[240,1],x[240,2],
x[260,0],x[260,1],x[260,2],
x[279,0],x[279,1],x[279,2],
x[298,0],x[298,1],x[298,2]]).reshape(4,3)
enseedPnts = np.array([x[397,0],x[397,1],x[397,2],
x[377,0],x[377,1],x[377,2],
x[365,0],x[365,1],x[365,2],
x[347,0],x[347,1],x[347,2],x[336,0],x[336,1],x[336,2]]).reshape(5,3)
esseedPnts = np.array([x[472,0],x[472,1],x[472,2],
x[462,0],x[462,1],x[462,2],
x[446,0],x[446,1],x[446,2],
x[427,0],x[427,1],x[427,2],x[417,0],x[417,1],x[417,2]]).reshape(5,3)

fracList = [f'OTP','West North','West South','West Deep','East North','East South']
nplanes = len(fracList)


In [ ]:
# write a function to fit a plane to the boundary points of the fracture
# this actually finds the normal vector to the plane
def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,n)
    points in d-dimensional space,
    fit a d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal vector to the plane, n.
    """
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trailing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    # M = np.dot(x, x.T) # Could also use np.cov(x) here.
    M = np.cov(x)
    normvec = svd(M)[0][:,-1]
    # do the dot product of the center point and the normal vector to find the offset
    fittedPlane = -ctr.dot(normvec)
    return ctr, normvec, fittedPlane



In [ ]:
# get the center and the normal vector to the planes
otpctr,otpnormvec,otpfittedPlane = planeFit(otp.T)
wnctr,wnnormvec,wnfittedPlane = planeFit(westN.T)
wsctr,wsnormvec,wsfittedPlane= planeFit(westS.T)
wdctr,wdnormvec,wdfittedPlane = planeFit(westD.T)
enctr,ennormvec,enfittedPlane  = planeFit(eastN.T)
esctr,esnormvec,esfittedPlane = planeFit(eastS.T)

# create points in the plane using the plane EQ
# Ax + By + Cz + D = 0 ... where A, B, C and D are the normal vector components and offset
# use x and z points in the plane to find the y points
# this works much better than using x and y to find z 
# y = -(Ax + Cz + D) / B
def makeplanepoints(seedPnts,normvec,fittedPlane):
    xb = np.sort(seedPnts[:,0].copy())
    zb = np.sort(seedPnts[:,2].copy())
    
    xx,zz = np.meshgrid(xb,zb)

    yy = -(normvec[0] * xx + normvec[2] * zz + fittedPlane) * 1. /normvec[1]

    return xx,yy,zz

otpxx,otpyy,otpzz = makeplanepoints(otp,otpnormvec,otpfittedPlane)
wnxx,wnyy,wnzz = makeplanepoints(westN,wnnormvec,wnfittedPlane)
wsxx,wsyy,wszz = makeplanepoints(westS,wsnormvec,wsfittedPlane)
wdxx,wdyy,wdzz = makeplanepoints(westD,wdnormvec,wdfittedPlane)
enxx,enyy,enzz = makeplanepoints(eastN,ennormvec,enfittedPlane)
esxx,esyy,eszz = makeplanepoints(eastS,esnormvec,esfittedPlane)

# code to make OTP bigger if we want to
xf = np.array([otpxx[i] for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2))])
yf = np.array([otpyy[i] for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2))])
zf = np.array([otpzz[i] for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2))])
corners = np.concatenate((xf[:,np.newaxis],yf[:,np.newaxis],zf[:,np.newaxis]),axis=1)
l1vec = corners[2,:] - corners[0,:]# type: ignore
l2vec = corners[1,:] - corners[3,:]# type: ignore
c1 =corners[0,:] - 0.5*(l1vec)
c2 =corners[1,:] + 0.5*(l2vec)
c3 =corners[0,:] + 1.5*(l1vec)
c4 =corners[1,:] - 1.5*(l2vec)
ncorners = np.vstack((c1,c2,c3,c4))
cnt=0
for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2)):
    otpxx[i] = ncorners[cnt,0]
    otpyy[i] = ncorners[cnt,1]
    otpzz[i] = ncorners[cnt,2]
    cnt+=1

In [ ]:
def fillPlane(x,y,z,numels):
    # find the corners of the plane using corner indices
    xf = np.array([x[i] for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2))])
    yf = np.array([y[i] for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2))])
    zf = np.array([z[i] for i in list(sliced((0,0,-1,0,-1,-1,0,-1),2))])

    corners = np.concatenate((xf[:,np.newaxis],yf[:,np.newaxis],zf[:,np.newaxis]),axis=1)
    # make points in between left,top,right, and bottom axes
    left = np.linspace(corners[0,:],corners[1,:],numels)
    top = np.linspace(corners[1,:],corners[2,:],numels)[1:-1]
    right = np.linspace(corners[3,:],corners[2,:],numels)
    bot = np.linspace(corners[0,:],corners[3,:],numels)[1:-1]
    # fill the interior points in the middle
    fp = []
    for i in range(numels-2):
        fp.append(np.linspace(bot[i,:],top[i,:],numels)[1:-1])
    # fp = np.array(fp).reshape(((numels-2)**2,corners.shape[1]))
    fp = np.array(fp)

    # build array correctly
    midarr = []
    for i in range(numels-2):
        midarr.append(np.vstack((bot[i,:],fp[i,:,:],top[i,:])))
    midarr = np.array(midarr).reshape((numels*(numels-2),3))

    return np.vstack((left,midarr,right))
    # return bot,fp,top

numels = [12,8,8,5,8,8]
otpsurf_pts = fillPlane(otpxx,otpyy,otpzz,numels[0])
wnsurf_pts = fillPlane(wnxx,wnyy,wnzz,numels[1])
wssurf_pts = fillPlane(wsxx,wsyy,wszz,numels[2])
wdsurf_pts = fillPlane(wdxx,wdyy,wdzz,numels[3])
ensurf_pts= fillPlane(enxx,enyy,enzz,numels[4])
essurf_pts = fillPlane(esxx,esyy,eszz,numels[5])


In [ ]:
# Create a mesh for Ben Thompson's strain matrix function in cutde
n_els_per_dim =[]
[n_els_per_dim.append(numels[el] - 1 ) for el in range(len(numels))]
allsurf_pts = [otpsurf_pts,wnsurf_pts,wssurf_pts,wdsurf_pts,ensurf_pts,essurf_pts]

def maketris(numels):
    surf_tris =[]
    nx = ny = numels + 1
    idx = lambda i, j: i * ny + j
    for i in range(numels):
        for j in range(numels):
            # x1, x2 = mesh_xs[i : i + 2]
            # y1, y2 = mesh_ys[j : j + 2]
            surf_tris.append([idx(i, j), idx(i + 1, j), idx(i + 1, j + 1)])
            surf_tris.append([idx(i, j), idx(i + 1, j + 1), idx(i, j + 1)])

    surf_tris = np.array(surf_tris, dtype=np.int64)
    return surf_tris

otpsurf_tris = maketris(n_els_per_dim[0])
wnsurf_tris = maketris(n_els_per_dim[1])
wssurf_tris = maketris(n_els_per_dim[2])
wdsurf_tris = maketris(n_els_per_dim[3])
ensurf_tris = maketris(n_els_per_dim[4])
essurf_tris = maketris(n_els_per_dim[5])

allsurf_tris = [otpsurf_tris,wnsurf_tris,wssurf_tris,wdsurf_tris,ensurf_tris,essurf_tris]


In [ ]:
# fig = go.Figure()
# # fig.add_scatter3d(x=np.array(isect)[:,0], y=np.array(isect)[:,1],z=np.array(isect)[:,2],mode='markers',
# #   marker=dict(
# #         size=8,
# #         color='purple',
# #         opacity=0.8))
# fig.add_scatter3d(x=pdtpts[:,0], y=pdtpts[:,1],z=pdtpts[:,2],mode='markers',
#   marker=dict(
#         size=3,
#         color=df_fullPDT[8462,:],
#         colorscale='rdbu',
#         cmid = 0,
#         opacity=0.8))
# fig.add_scatter3d(x=pdbpts[:,0], y=pdbpts[:,1],z=pdbpts[:,2],mode='markers',
#   marker=dict(
#         size=3,
#         color=df_fullPDB[8462,:],
#         colorscale='rdbu',
#         cmid = 0,
#         opacity=0.8))
# fig.add_scatter3d(x=pstpts[:,0], y=pstpts[:,1],z=pstpts[:,2],mode='markers',
#   marker=dict(
#         size=3,
#         color=df_fullPST[8462,:],
#         colorscale='rdbu',
#         cmid = 0,
#         opacity=0.8))
# fig.add_scatter3d(x=psbpts[:,0], y=psbpts[:,1],z=psbpts[:,2],mode='markers',
#   marker=dict(
#         size=3,
#         color=df_fullPSB[8462,:],
#         colorscale='rdbu',
#         cmid = 0,
#         opacity=0.8))
# fig.add_scatter3d(x=obpts[:,0], y=obpts[:,1],z=obpts[:,2],mode='markers',
#   marker=dict(
#         size=3,
#         color=df_fullOB[8462,:],
#         colorscale='rdbu',
#         cmid = 0,
#         opacity=0.8))
# fig.add_scatter3d(x=otpts[:,0], y=otpts[:,1],z=otpts[:,2],mode='markers',
#   marker=dict(
#         size=3,
#         color=df_fullOT[8462,:],
#         colorscale='rdbu',
#         cmid = 0,
#         colorbar=dict(thickness=20),
#         opacity=0.8))
# fig.add_trace(go.Mesh3d(x=otp[:,0],y=otp[:,1],z=otp[:,2],
#         opacity=0.5,
#         color='grey',
#         text=["OT-P Connector"],
#         ))
# fig.add_trace(go.Mesh3d(x=westN[:,0],y=westN[:,1],z=westN[:,2],
#         opacity=0.5,
#         color='grey',
#         text=["WN"],
#         ))
# fig.add_trace(go.Mesh3d(x=westS[:,0],y=westS[:,1],z=westS[:,2],
#         opacity=0.5,
#         color='grey',
#         text=["WS"],
#         ))
# fig.add_trace(go.Mesh3d(x=westD[:,0],y=westD[:,1],z=westD[:,2],
#         opacity=0.5,
#         color='grey',
#         text=["WD"],
#         ))
# fig.add_trace(go.Mesh3d(x=eastN[:,0],y=eastN[:,1],z=eastN[:,2],
#         opacity=0.5,
#         color='grey',
#         text=["EN"],
#         ))
# fig.add_trace(go.Mesh3d(x=eastS[:,0],y=eastS[:,1],z=eastS[:,2],
#         opacity=0.5,
#         color='grey',
#         text=["ES"],
#         ))
# # fig.add_trace(go.Mesh3d(x=westN[:,0],y=westN[:,1],z=westN[:,2],
# #         opacity=0.5,
# #         color='magenta',
# #         text=["wncircle"],
# #         ))
# # fig.add_scatter3d(x=otpsurf_pts[:,0], y=otpsurf_pts[:,1],z=otpsurf_pts[:,2],mode='markers',
# #   marker=dict(
# #         size=3,
# #         color='pink',
# #         opacity=0.7))
# # fig.add_scatter3d(x=ensurf_pts[:,0], y=ensurf_pts[:,1],z=ensurf_pts[:,2],mode='markers',
# #   marker=dict(
# #         size=3,
# #         color='green',
# #         opacity=0.4))
# fig.update_layout(
#         autosize=False,
#         width=1200,
#         height=800,
#         )


# fig.show()

In [ ]:
# Create figure
fig = go.Figure(
    data=[go.Scatter3d(x=pdtpts[:,0], y=pdtpts[:,1],z=pdtpts[:,2],mode='markers',
        marker=dict(
            size=3,
            color=df_fullPDT[0,:pdtpts.shape[0]],
            colorscale='rdbu',
            cmid = 0,
            opacity=0.8)),
          go.Scatter3d(x=pdbpts[:,0], y=pdbpts[:,1],z=pdbpts[:,2],mode='markers',
        marker=dict(
            size=3,
            color=df_fullPDB[0,:],
            colorscale='rdbu',
            cmid = 0,
            opacity=0.8)),
        go.Scatter3d(x=psbpts[:,0], y=psbpts[:,1],z=psbpts[:,2],mode='markers',
        marker=dict(
            size=3,
            color=df_fullPSB[0,:],
            colorscale='rdbu',
            cmid = 0,
            opacity=0.8)),
        go.Scatter3d(x=pstpts[:,0], y=pstpts[:,1],z=pstpts[:,2],mode='markers',
        marker=dict(
            size=3,
            color=df_fullPST[0,:],
            colorscale='rdbu',
            cmid = 0,
            opacity=0.8)),
        go.Scatter3d(x=otpts[:,0], y=otpts[:,1],z=otpts[:,2],mode='markers',
        marker=dict(
            size=3,
            color=df_fullOT[0,:otpts.shape[0]],
            colorscale='rdbu',
            cmid = 0,
            opacity=0.8)),
        go.Scatter3d(x=obpts[:,0], y=obpts[:,1],z=obpts[:,2],mode='markers',
        marker=dict(
            size=3,
            color=df_fullOB[0,:],
            colorscale='balance',
            cmid = 0,
            opacity=0.8)),
                     ],
    layout=go.Layout(
        title_text=f'Strain Rates', hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter3d(
            x=pdtpts[:,0],
            y=pdtpts[:,1],
            z=pdtpts[:,2],
            mode='markers',
            marker=dict(
            size=3,
            color=df_fullPDT[k,:otpts.shape[0]],
            colorscale='rdbu',
            cmid = 0,
            colorbar=dict(thickness=20),
            opacity=1)),
            ],layout=go.Layout(title_text=f'Strain Rates {DASdates[k]}'))
        for k in np.arange(5550,df_fullOT.shape[0],50)]


)
fig.add_trace(go.Mesh3d(x=otp[:,0],y=otp[:,1],z=otp[:,2],
        opacity=0.25,
        color='grey',
        text=["OT-P Connector"],
        ))
fig.add_trace(go.Mesh3d(x=westN[:,0],y=westN[:,1],z=westN[:,2],
        opacity=0.25,
        color='grey',
        text=["WN"],
        ))
fig.add_trace(go.Mesh3d(x=westS[:,0],y=westS[:,1],z=westS[:,2],
        opacity=0.25,
        color='grey',
        text=["WS"],
        ))
fig.add_trace(go.Mesh3d(x=westD[:,0],y=westD[:,1],z=westD[:,2],
        opacity=0.25,
        color='grey',
        text=["WD"],
        ))
fig.add_trace(go.Mesh3d(x=eastN[:,0],y=eastN[:,1],z=eastN[:,2],
        opacity=0.25,
        color='grey',
        text=["EN"],
        ))
fig.add_trace(go.Mesh3d(x=eastS[:,0],y=eastS[:,1],z=eastS[:,2],
        opacity=0.25,
        color='grey',
        text=["ES"],
        ))
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
)
fig.show()

In [ ]:
# Call strain matrix function from Ben's FullSpace code 
# output dimensions are strain_mat = (num_obs, strain_tensor_component, num_triangles, slip mode)
# strain tensor comps: 0 = xx, 1 = yy, 2 = zz, 3 = xy, 4 = xz, 5 = yz
# slip modes: 0 = strike slip, 1 = dip slip, 2 = tensile

allstrain_mat = FS.strain_matrix(obs_pts=allpts, tris=np.concatenate([allsurf_pts[i][tri] for i,tri in enumerate(allsurf_tris)]),nu=0.24)

In [ ]:
def genSlice(sliceLen,nummodes):
    '''Generate a list of slices
    for grabbing the number of 
    triangles in the mesh for each slip mode'''

    first = int(sliceLen / nummodes)
    second = int(first * 2)
    third  = int(sliceLen)
    
    lst = list(sliced((int(0),first,first,second,second,third),2))
    return lst
def fracSlice(start,numfracs):
    '''Generate a list of slices
    for grabbing the number of 
    triangles in the mesh for each indivial fracture'''
    first = start + 2*( numfracs[0]**2 )
    second = first + 2*( numfracs[1]**2 ) 
    third  = second + 2*( numfracs[2]**2 ) 
    fourth = third + 2*( numfracs[3]**2 ) 
    fifth = fourth + 2*( numfracs[4]**2 ) 
    sixth = fifth + 2*( numfracs[5]**2 )
    
    lst = list(sliced((start,first,first,second,second,third,third,fourth,fourth,fifth,fifth,sixth),2))
    return lst

# slices for splitting the model estimates into their respective modes (strike, dip, tensile)
mode = genSlice(allstrain_mat.shape[2]*allstrain_mat.shape[3],3)
# slices for splitting elements into their respective fracture plane
ifrac = fracSlice(int(mode[0][0]),n_els_per_dim)
jfrac = fracSlice(int(mode[1][0]),n_els_per_dim)
kfrac = fracSlice(int(mode[2][0]),n_els_per_dim)
mfrac = list(zip(ifrac,jfrac,kfrac))

In [ ]:
# Function for finding tangent vectors along wells to use for rotating strains 

def getRotVecs(points):
    # points in N x 3 
    # convert points to array if list or dataframe
    points = np.array(points)
    # find the derivative of the original points (tangent)
    d_points = np.gradient(points,axis=0)
    # divide by length to get unit tangent
    tangents = d_points / norm(d_points,ord=2)
    # find the derivative of the tangents to get normal
    # d_tans = np.gradient(tangents,axis=1)
    # # divide by length to get unit normal
    # normals = d_tans / norm(d_tans,ord=2)
    # # take cross product to get bi normal
    # bnorms = np.cross(tangents,normals)
    return tangents
    # return tangents,normals,bnorms

pdttans = getRotVecs(pdtpts)
pdbtans = getRotVecs(pdbpts)
psttans = getRotVecs(pstpts)
psbtans = getRotVecs(psbpts)
ottans = getRotVecs(otpts)
obtans = getRotVecs(obpts)

In [7]:
# Function for doing the strain rotations given the tangent vectors and strains 
# ( See Liam Chiang / Paul Sava paper " High resolution multi-component DAS" )
# Equations in Appendix

def rotStrains(tanvecs,strains):
    R_11 = tanvecs[:,0]
    R_12 = tanvecs[:,1]
    R_13 = tanvecs[:,2]
    
    rotmat = np.zeros((strains.shape[0],strains.shape[2],strains.shape[3]))
    G = np.array([R_11**2, R_12**2, R_13**2, 2*R_11*R_12, 2*R_11*R_13, 2*R_12*R_13]).T
    for i in range(strains.shape[3]):
        for j in range(strains.shape[0]):
            
            rotmat[j,:,i] = G[j,:] @ strains[j,:,:,i]
    return rotmat

first = len(pdtpts)
second = len(pdtpts)+len(pdbpts)
third = second + len(pstpts)
fourth = third + len(psbpts)
fifth = fourth + len(otpts)
sixth = fifth + len(obpts)
lenvec = list(sliced((int(0),first,first,second,second,third,third,fourth,fourth,fifth,fifth,sixth),2))

# Do the rotations

pdtstrain_mat_rot = rotStrains(pdttans,allstrain_mat[lenvec[0][0]:lenvec[0][1],:,:,:])
pdbstrain_mat_rot = rotStrains(pdbtans,allstrain_mat[lenvec[1][0]:lenvec[1][1],:,:,:])
pststrain_mat_rot = rotStrains(psttans,allstrain_mat[lenvec[2][0]:lenvec[2][1],:,:,:])
psbstrain_mat_rot = rotStrains(psbtans,allstrain_mat[lenvec[3][0]:lenvec[3][1],:,:,:])
otstrain_mat_rot = rotStrains(ottans,allstrain_mat[lenvec[4][0]:lenvec[4][1],:,:,:])
obstrain_mat_rot = rotStrains(obtans,allstrain_mat[lenvec[5][0]:lenvec[5][1],:,:,:])

# Combine the rotated strain matrices and make the G matrix for inversion
allstrain_mat_rot = np.concatenate((pdtstrain_mat_rot,pdbstrain_mat_rot,pststrain_mat_rot,psbstrain_mat_rot,otstrain_mat_rot,obstrain_mat_rot))
allG = np.hstack((allstrain_mat_rot[:,:,0],allstrain_mat_rot[:,:,1],allstrain_mat_rot[:,:,2]))

In [ ]:
# Definitions for Ridge Regression 

def ridge(X, y, l2):
    """Ridge Regression model with intercept term.
    L2 penalty and intercept term included via design matrix augmentation.
    Params:
        X - NumPy matrix, size (N, p), of numerical predictors
        y - NumPy array, length N, of numerical response
        l2 - L2 penalty tuning parameter (positive scalar) 
    Returns:
        NumPy array, length p + 1, of fitted model coefficients
    """
    m, n = np.shape(X)
    upper_half = np.hstack((np.ones((m, 1)), X))
    # upper_half = X
    lower = np.zeros((n, n))
    np.fill_diagonal(lower, np.sqrt(l2))
    lower_half = np.hstack((np.zeros((n, 1)), lower))
    # lower_half = lower
    X = np.vstack((upper_half, lower_half))
    y = np.append(y, np.zeros(n))
    if m == n:
        soln = solve(np.dot(X.T, X), np.dot(X.T, y))
    else:
        soln = lstsq(np.dot(X.T, X), np.dot(X.T, y))
        U,s,Vt = svd(np.dot(X.T, X),lapack_driver='gesvd')
        # soln = lstsq(X, y)
        # U,s,Vt = svd(X)
        
    return soln, U, s, Vt, X, y

def getCond(X, y, l2):
    """ Find condition number of augmented matrix for Ridge 
    L2 penalty and intercept term included via design matrix augmentation.
    Params:
        X - NumPy matrix, size (N, p), of numerical predictors
        y - NumPy array, length N, of numerical response
        l2 - L2 penalty tuning parameter (positive scalar) 
    Returns:
        NumPy array, length p + 1, of fitted model coefficients
    """
    m, n = np.shape(X)
    upper_half = np.hstack((np.ones((m, 1)), X))
    lower = np.zeros((n, n))
    np.fill_diagonal(lower, np.sqrt(l2))
    lower_half = np.hstack((np.zeros((n, 1)), lower))
    X = np.vstack((upper_half, lower_half))
    
    condition_num = np.linalg.cond(np.dot(X.T, X))
    return condition_num

In [ ]:
das_time_ind = 5785 # 2018-05-24 22:38:09
# das_time_ind = 7782 # 2018-05-25 15:18:48
# das_time_ind = 8462 # 2018-05-25 20:58:48
print(DASdates[das_time_ind])


In [ ]:
pdtdat = df_strainPDT[das_time_ind,:pdtpts.shape[0]] 
pdbdat = df_strainPDB[das_time_ind,:pdbpts.shape[0]] 
pstdat = df_strainPST[das_time_ind,:pstpts.shape[0]] 
psbdat = df_strainPSB[das_time_ind,:psbpts.shape[0]] 
otdat = df_strainOT[das_time_ind,:otpts.shape[0]] 
obdat = df_strainOB[das_time_ind,:obpts.shape[0]] 

pdtdat1 = df_strainPDT[das_time_ind,:pdtpts.shape[0]] 
pdbdat1 = df_strainPDB[das_time_ind,:pdbpts.shape[0]] 
pstdat1 = df_strainPST[das_time_ind,:pstpts.shape[0]] 
psbdat1 = df_strainPSB[das_time_ind,:psbpts.shape[0]] 
otdat1 = df_strainOT[das_time_ind,:otpts.shape[0]] 
obdat1 = df_strainOB[das_time_ind,:obpts.shape[0]] 

pdtdat = pdtdat - (1.056e-5 * pdtdts[:pdtpts.shape[0]]) * 1e9
pdbdat = pdbdat - (1.056e-5 * pdbdts[:pdbpts.shape[0]]) * 1e9 
pstdat = pstdat - (.0561e-5 * pstdts[:pstpts.shape[0]]) * 1e9
psbdat = psbdat - (1.056e-5 * psbdts[:psbpts.shape[0]]) * 1e9
otdat  = otdat - (1.056e-5 * otdts[:otpts.shape[0]]) * 1e9 
obdat  = obdat - (1.056e-5 * obdts[:obpts.shape[0]]) * 1e9 
Datcomb = np.hstack((detrend(pdtdat,type='constant'),
                     detrend(pdbdat,type='constant'),
                     detrend(pstdat,type='constant'),
                     detrend(psbdat,type='constant'),
                     detrend(otdat,type='constant'),
                     detrend(obdat,type='constant')))
Datcomb1 = np.hstack((pdtdat1,pdbdat1,pstdat1,psbdat1,otdat1,obdat1))


In [ ]:
pdttemp= (1.056e-5 * pdtdts[:pdtpts.shape[0]]) * 1e9
pdbtemp = (1.056e-5 * pdbdts[:pdbpts.shape[0]]) * 1e9 
psttemp = (.0561e-5 * pstdts[:pstpts.shape[0]]) * 1e9
psbtemp= (1.056e-5 * psbdts[:psbpts.shape[0]]) * 1e9
ottemp  = (1.056e-5 * otdts[:otpts.shape[0]]) * 1e9 
obtemp = (1.056e-5 * obdts[:obpts.shape[0]]) * 1e9 
plt.figure()
plt.plot(pdttemp,label='pdt')
plt.plot(pdbtemp,label='pdb')
plt.plot(psttemp,label='pst')
plt.plot(psbtemp,label='psb')
plt.plot(ottemp,label='ot')
plt.plot(obtemp,label='ob')
plt.xlabel('Channel')
plt.ylabel('Thermo-optical Strain')
plt.legend()

stdstack = np.std(np.vstack([pdttemp,pdbtemp,psbtemp,ottemp,obtemp]))

In [ ]:
tick_labels=['30','\n\nPDT','90','\n\nPDB','144','\n\nPST','194','\n\nPSB','254','\n\nOT','314','\n\nOB']

tick_locations = np.array((30,90,144,194,254,314))
ptLen = np.cumsum((pdtpts.shape[0],pdbpts.shape[0],pstpts.shape[0],psbpts.shape[0],otpts.shape[0],obpts.shape[0]))
new_labels = [ ''.join(x) for x in zip(tick_labels[0::2], tick_labels[1::2]) ]
fig,ax = plt.subplots(figsize=(10,8))
ax.plot(Datcomb,label='temp corrected')
ax.fill_between(range(len(Datcomb)),Datcomb+stdstack,Datcomb-stdstack,color='black',label='temp corrected')
# ax.fill_between(Datcomb-stdstack,color='black',label='temp corrected')
ax.plot(Datcomb1,label='non-temp corrected')
ax.set_ylabel('Strain')
ax.set_xlabel('Well Center')
ax.set_xticks(tick_locations, new_labels)
ax.axvspan(0,ptLen[0],facecolor='green',alpha = 0.2)
ax.axvspan(ptLen[0],ptLen[1],facecolor='yellow',alpha = 0.2)
ax.axvspan(ptLen[1],ptLen[2],facecolor='red',alpha = 0.2)
ax.axvspan(ptLen[2],ptLen[3],facecolor='blue',alpha = 0.2)
ax.axvspan(ptLen[3],ptLen[4],facecolor='orange',alpha = 0.2)
ax.axvspan(ptLen[4],ptLen[5],facecolor='grey',alpha = 0.2)
ax.set_title('DAS Data Temp Corrections')

plt.legend()

In [ ]:


plt.figure()
lam = np.logspace(-13,-3,25)
cn = np.zeros_like(lam)
sn = np.zeros_like(lam)
SSE = np.zeros_like(lam)
var_res = np.zeros_like(lam)
# MSE = np.zeros_like(lam)
for i in range(len(lam)):
    # lam = 0.000001
    testmest,_,_,_,X,y = ridge(allG,Datcomb1,lam[i])
    pred_dat = X @ testmest[0]
    # print(np.mean(Datcomb-pred_dat[:349]))
    # print(np.std(Datcomb-pred_dat[:349]))
    # cn[i] = getCond(allG,Datcomb,lam[i])
    var_res[i] = np.var((X @ testmest[0] - y ))
    # SSE[i] = np.sum(((Datcomb - allG @ testmest[0]))**2)
    SSE[i] = norm(X @ testmest[0] - y)
    sn[i] = norm(testmest[0])
    # MSE[i] = np.mean(((allG @ testmest[0][1:]) - Datcomb)**2)
# plt.semilogx(lam,MSE,'.r',label='MSE')
# plt.semilogx(lam,SSE,'.g',label='SSE')
# plt.loglog(lam,cn)
plt.loglog(SSE,sn,'.')
# plt.xlabel('log(Lambda)')
plt.xlabel('log(Residual)')
# plt.ylabel('Matrix Condition Number')
plt.ylabel('log(norm(solutions))')
# plt.title('log(Lambda) vs Condition Number')
plt.title('L-curve')
plt.legend()
print(lam)

In [ ]:
lam = np.logspace(-13,-3,25)
fig,ax = plt.subplots()
ax2= ax.twinx()
fig.suptitle('Bias / Variance Tradeoff')
ax2.loglog(lam,sn,label='solution norm')
ax2.set_ylabel('solution norm')
ax2.legend(loc='center right')
ax.loglog(lam,SSE,color='orange',label='residual norm ')
ax.set_ylabel('norm of residual')
# ax.set_ylim(4e3,1e5)
ax.legend(loc='center left')
ax.set_xlabel('Lambda')

In [ ]:

# lam = 3.5e-11
# lam = 3.8e-9
# lam = 8.9e-8
lam = 1e-8
lamT = 4.2e-8

# pdtdat = df_strainPDT[das_time_ind,:pdtpts.shape[0]] 
# pdbdat = df_strainPDB[das_time_ind,:pdbpts.shape[0]] 
# pstdat = df_strainPST[das_time_ind,:pstpts.shape[0]] 
# psbdat = df_strainPSB[das_time_ind,:psbpts.shape[0]] 
# otdat = df_strainOT[das_time_ind,:otpts.shape[0]] 
# obdat = df_strainOB[das_time_ind,:obpts.shape[0]] 
# Datcomb = np.hstack((pdtdat,pdbdat,pstdat,psbdat,otdat,obdat))

mest,U,s,Vt, G, daty = ridge(allG, Datcomb1,lam)
mestTemp,Utemp,stemp,Vtemp, Gtemp, datytemp = ridge(allG, Datcomb,lamT)

# U = U[:-1,:-1]
# s = s[:-1]
# Vt = Vt[:-1,:-1]
# res = mest[1]
mest = mest[0][1:]
mestTemp = mestTemp[0][1:]
# daty = daty[:len(mest)]

# G = G[:,:-1]



In [ ]:
# Do the inverse problem for several time slices
# lam = 6.9e-8
# strainList = [df_strainPDT,df_strainPDB,df_strainPST,df_strainPSB,df_strainOT,df_strainOB]
# pntShapeList = [pdtpts.shape[0],pdbpts.shape[0],pstpts.shape[0],psbpts.shape[0],otpts.shape[0],obpts.shape[0]]
# das_time_ind = np.arange(7765,7816,1)
# def doInverse(timeEpochs,strainList,Gmat,lam,pntShapeList):
#     mest = []
#     for i in timeEpochs:
#         pdtdat = strainList[0][i,:pdtpts.shape[0]] 
#         pdbdat = strainList[1][i,:pdbpts.shape[0]] 
#         pstdat = strainList[2][i,:pstpts.shape[0]] 
#         psbdat = strainList[3][i,:psbpts.shape[0]] 
#         otdat = strainList[4][i,:otpts.shape[0]] 
#         obdat = strainList[5][i,:obpts.shape[0]]
        
#         pdtdat = pdtdat - (1.056e-5 * pdtdts[:pdtpts.shape[0]]) * 1e9
#         pdbdat = pdbdat - (1.056e-5 * pdbdts[:pdbpts.shape[0]]) * 1e9 
#         pstdat = pstdat - (.0561e-5 * pstdts[:pstpts.shape[0]]) * 1e9
#         psbdat = psbdat - (1.056e-5 * psbdts[:psbpts.shape[0]]) * 1e9
#         otdat  = otdat - (1.056e-5 * otdts[:otpts.shape[0]]) * 1e9 
#         obdat  = obdat - (1.056e-5 * obdts[:obpts.shape[0]]) * 1e9 
#         Datcomb = np.hstack((detrend(pdtdat,type='constant'),
#                      detrend(pdbdat,type='constant'),
#                      detrend(pstdat,type='constant'),
#                      detrend(psbdat,type='constant'),
#                      detrend(otdat,type='constant'),
#                      detrend(obdat,type='constant')))
#         tmp,_,_,_,_,_ = ridge(Gmat,Datcomb,lam)
#         mest.append(tmp[0][1:])
#     return mest

# mest = doInverse(das_time_ind,strainList,allG,lam,pntShapeList)
# plt.figure()
# for i in range(len(mest)):
#     plt.plot(allG@mest[i])

In [ ]:
U,s,Vt = svd(allG,full_matrices=False)
# G1 = Vt.T @ np.linalg.inv(np.diag(s)) @ U.T
G1 = Vt[:100,:].T @ np.linalg.inv(np.diag(s[:100])) @ U[:,:100].T
mest1 = G1 @ Datcomb1

In [ ]:
fig,ax = plt.subplots(1,2)
ax[0].plot(mest,label='Ridge')
ax[0].legend()
ax[1].plot(mest1,label='SVD')
ax[1].legend()

In [ ]:
fig,ax = plt.subplots(1,2)
fig.suptitle('Data fits: predicted v. observed')
ax[0].plot(allG @ mest,label='ridge')
ax[0].set_ylabel('model estimates')
ax[0].set_xlabel('data index (well) ')
ax[0].plot(Datcomb1,label='observed')
ax[1].plot(allG @ mest1,label='svd')
ax[1].set_xlabel('data index (well) ')
ax[1].plot(Datcomb1,label='observed')
ax[0].legend()
ax[1].legend()
fig.tight_layout()


In [ ]:
pred_dat1 = allG @ mest1
pred_dat = allG @ mest
print(f'Ridge resid mean is {np.mean(Datcomb1-pred_dat)}')
print(f'Ridge resid std is {np.std(Datcomb1-pred_dat)}')
print(f'SVD resid mean is {np.mean(Datcomb1-pred_dat1)}')
print(f'SVD resid std is {np.std(Datcomb1-pred_dat1)}')
print(f'temp corrected Ridge resid mean is {np.mean(Datcomb-pred_dat)}')
print(f'temp correctedRidge resid std is {np.std(Datcomb-pred_dat)}')
print(f'temp correctedSVD resid mean is {np.mean(Datcomb-pred_dat1)}')
print(f'temp correctedSVD resid std is {np.std(Datcomb-pred_dat1)}')

In [ ]:
# Calculate data and model resolution

Gg = np.linalg.inv(G[:,1:].T @ G[:,1:] ) @ allG.T 
var_res = np.var((daty - G[:,1:] @ mest ))
W = np.linalg.inv(G[:,1:].T @ G[:,1:] ) @ (allG.T @ allG)
covm = var_res * W @ np.linalg.inv(allG.T @ allG) @ W.T

In [ ]:
fig,ax = plt.subplots(1,2)
im1 = ax[0].plot(np.diag(covm))
im2 = ax[1].imshow(covm, aspect = 'auto')
ax[1].set_aspect('equal',adjustable='box')
cbar1 = plt.colorbar(im2, ax=ax[1],label='Model Covariance')
ax[1].invert_yaxis()
# im3 = ax[2].imshow(covd, aspect = 'auto',vmin=0,vmax=0.1)
# cbar2 = plt.colorbar(im3, ax=ax[2],label='Data Covariance')
# im4 = ax[3].imshow(covu, aspect = 'auto')
# cbar3 = plt.colorbar(im4, ax=ax[3],label='Unit Model Covariance')
fig.tight_layout()


In [ ]:

titles=[f'Strike Slip ','Dip Slip','Tensile ']
fig,ax = plt.subplots(nplanes,3,figsize=(12,10))
for i in range(3):
    for k in range(nplanes):

        # vm = np.percentile(mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],90)
        vm = np.percentile(mest[mode[i][0]:mode[i][1]],99)
        # covvm = np.percentile(modcov[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],99)
        # vm = np.percentile(mest[mode[i][0]:mode[i][1]],99)


        # im  = ax[k,i].tripcolor(allsurf_pts[pfrac[k][0]:pfrac[k][1], 0], allsurf_pts[pfrac[k][0]:pfrac[k][1], 2],
        #                         allsurf_tris[ifrac[k][0]:ifrac[k][1]],
        #                         facecolors=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #                         cmap='seismic',vmin=-50000,vmax=50000) # type: ignore
        
        im  = ax[k,i].tripcolor(np.array(allsurf_pts[k])[:,0], np.array(allsurf_pts[k])[:,2],
                        np.array(allsurf_tris[k]),
                        facecolors=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
                        cmap='seismic',
                        vmin = -vm,
                        vmax = vm)
                        # vmin=min(mest[mode[i][0]:mode[i][1]]),
                        # vmax=max(mest[mode[i][0]:mode[i][1]])) # type: ignore
        # im  = ax[k,i].tripcolor(np.array(allsurf_pts[k])[:,0], np.array(allsurf_pts[k])[:,2],
        #         np.array(allsurf_tris[k]),
        #         facecolors=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #         cmap='cmr.lilac_r') # type: ignore
        # im  = ax[k,i].tripcolor(np.array(allsurf_pts[k])[:,0], np.array(allsurf_pts[k])[:,2],
        #         np.array(allsurf_tris[k]),
        #         facecolors=modcov[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #         cmap='cmr.lilac_r',vmin=94000,vmax=95000) # type: ignore
        ax[k,i].set_xlabel('X direction (m)')
        ax[k,i].set_ylabel('Z direction (m)')
        # ax[i].axis('equal')
        # ax[i].set_aspect('equal','box')
        plt.colorbar(im,ax=ax[k,i])
        ax[k,i].set_title(f'{fracList[k]} {titles[i]}')
        fig.tight_layout()

In [ ]:
# combine model estimates over each mode of slip to get total slip magnitude

ss = []
ds = []
ts = []
mode = genSlice(len(mest),3)
ifrac = fracSlice(int(mode[0][0]),n_els_per_dim)
for k in range(nplanes):

    ss.append(mest[mode[0][0]:mode[0][1]][ifrac[k][0]:ifrac[k][1]])
    ds.append(mest[mode[1][0]:mode[1][1]][ifrac[k][0]:ifrac[k][1]])
    ts.append(mest[mode[2][0]:mode[2][1]][ifrac[k][0]:ifrac[k][1]])

zipped_list = list(zip(ss,ds,ts))
allslip = []
i=0
for frac in range(len(zipped_list)):
    tmp2 = []
    for elem in range(len(zipped_list[frac][i])):
        tmp = []
        for mode in range(len(zipped_list[0])):
            tmp.append(zipped_list[frac][mode][elem]**2)
        tmp2.append(np.sqrt(sum(tmp)))
        i =+ 1
    allslip.append(tmp2)

In [ ]:
# Look at some confidence intervals
# calc standard devs
modcov = np.diag(covm)
frac_var = []
frac_stds = []
maxslip = []
maxslipInd = []
covfracs = []
mode = genSlice(len(mest),3)
ifrac = fracSlice(int(mode[0][0]),n_els_per_dim)
for i in range(len(mode)):
    for k in range(nplanes):
        covfracs.append(modcov[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]])
        frac_stds.append(np.std(mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]]))
        frac_var.append(np.var(mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]]))
        maxslip.append(np.max(np.abs(mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]])))
        maxslipInd.append(np.argmax(np.abs(mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]])))
        print(f'maxslip is {maxslip[k]}')
        print(f'maxslipInd is {maxslipInd[k]}')



In [ ]:
# CI for highest slip on each fracture for each mode

maxslip_CI = []
maxslip_CIstd = []
# mode = genSlice(len(mest),3)
# ifrac = fracSlice(int(mode[0][0]),mode[0][1],nplanes)
for i in range(len(maxslip)):
    maxslip_CI.append((maxslip[i] - covfracs[i][maxslipInd[i]], maxslip[i] + covfracs[i][maxslipInd[i]]))
    maxslip_CIstd.append((maxslip[i] - frac_stds[i], maxslip[i] + frac_stds[i]))
print(maxslip_CI)
print(maxslip_CIstd)

In [ ]:
# CIs for integrated slip on each fracture plane

# mode = genSlice(len(mest),3)
# ifrac = fracSlice(int(mode[0][0]),mode[0][1],nplanes)
# avg_open = []
# for i in range(len(mode)):
#     for k in range(nplanes):
#         sumvar = np.sum(modcov[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]])
#         avg_open.append(sumvar/num_tri_frac)
fracind = list(sliced((0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5),1))
num_tri_frac = [2*(els**2) for els in n_els_per_dim]
avg_open = []
for i in range(len(maxslip_CI)):
    sumvar = maxslip_CI[i][0] + maxslip_CI[i][1]
    avg_open.append(sumvar/num_tri_frac[fracind[i][0]])


In [ ]:
# Average opening on each fault plane for each mode in microns
[print(avg_open[i]/1e3) for i in range(len(avg_open))]

In [ ]:
plot_dat = mest
mode = genSlice(len(mest),3)
ifrac = fracSlice(int(mode[0][0]),n_els_per_dim)
for i in range(len(mode)):
        fig = go.Figure(data=[go.Scatter3d(x=meq.x,y=meq.y,z=meq.z,mode='markers'
                                        ,marker=dict(size=3,color=meq.datenums,colorscale='RdBu',
                                                ))])
        fig.add_scatter3d(x=swells[swells["HoleID"]=='E1-I'].x, # type: ignore
                           y=swells[swells["HoleID"]=='E1-I'].y, # type: ignore
                           z=swells[swells["HoleID"]=='E1-I'].z, # type: ignore
                           mode='lines',
        marker=dict(
                size=3,
                color='black',
                opacity=0.8))
        fig.add_scatter3d(x=swells[swells["HoleID"]=='E1-P'].x, # type: ignore
                          y=swells[swells["HoleID"]=='E1-P'].y, # type: ignore
                          z=swells[swells["HoleID"]=='E1-P'].z, # type: ignore
                          mode='lines',
        marker=dict(
                size=3,
                color='black',
                opacity=0.8))
        fig.add_scatter3d(x=pdtpts[:,0], y=pdtpts[:,1],z=pdtpts[:,2],mode='lines',
        marker=dict(
                size=3,
                color='red',
                opacity=0.8))
        fig.add_scatter3d(x=pdbpts[:,0], y=pdbpts[:,1],z=pdbpts[:,2],mode='lines',
        marker=dict(
                size=3,
                color='red',
                opacity=0.8))
        fig.add_scatter3d(x=pstpts[:,0], y=pstpts[:,1],z=pstpts[:,2],mode='lines',
        marker=dict(
                size=3,
                color='blue',
                opacity=0.8))
        fig.add_scatter3d(x=psbpts[:,0], y=psbpts[:,1],z=psbpts[:,2],mode='lines',
        marker=dict(
                size=3,
                color='blue',
                opacity=0.8))
        fig.add_scatter3d(x=obpts[:,0], y=obpts[:,1],z=obpts[:,2],mode='lines',
        marker=dict(
                size=3,
                color='green',
                opacity=0.8))
        fig.add_scatter3d(x=otpts[:,0], y=otpts[:,1],z=otpts[:,2],mode='lines',
        marker=dict(
                size=3,
                color='green',
                opacity=0.8))
        for k in range(nplanes):
                # normV = Normalize(vmin=0,vmax=10000)
                fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
                i=np.array(allsurf_tris[k])[:,0],j=np.array(allsurf_tris[k])[:,1],k=np.array(allsurf_tris[k])[:,2],
                colorscale='thermal',facecolor=plot_dat[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
                intensity=plot_dat[mode[i][0]:mode[i][1]],
                intensitymode='cell'))
        # for k in range(nplanes):
        #         # normV = Normalize(vmin=0,vmax=10000)
        #         fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
        #         i=np.array(allsurf_tris[k])[:,0],j=np.array(allsurf_tris[k])[:,1],k=np.array(allsurf_tris[k])[:,2],
        #         colorscale='thermal',facecolor=plot_dat[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #         intensity=plot_dat[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #         intensitymode='cell',
        #         cmin=np.min(np.array([plot_dat[mfrac[k][0][0]:mfrac[k][0][1]]] + [plot_dat[mfrac[k][1][0]:mfrac[k][1][1]]] + [plot_dat[mfrac[k][2][0]:mfrac[k][2][1]]])),
        #         cmax=np.max(np.array([plot_dat[mfrac[k][0][0]:mfrac[k][0][1]]] + [plot_dat[mfrac[k][1][0]:mfrac[k][1][1]]] + [plot_dat[mfrac[k][2][0]:mfrac[k][2][1]]]))))
        # for k in range(nplanes):
        #         # normV = Normalize(vmin=0,vmax=10000)
        #         fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
        #         i=np.array(allsurf_tris[k])[:,0],j=np.array(allsurf_tris[k])[:,1],k=np.array(allsurf_tris[k])[:,2],
        #         colorscale='thermal',facecolor=plot_dat[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #         intensity=plot_dat[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
        #         intensitymode='cell',
        #         cmin=-np.percentile(plot_dat[mode[i][0]:mode[i][1]],99),
        #         cmax=np.percentile(plot_dat[mode[i][0]:mode[i][1]],99)))
        fig.update_layout(
                autosize=False,
                width=1200,
                height=800,)
        fig.show()

In [ ]:


minslip = []
maxslip = []
[minslip.append(min(allslip[i])) for i in range(len(allslip))]
[maxslip.append(max(allslip[i])) for i in range(len(allslip))]


fig = go.Figure(data=[go.Scatter3d(x=meq.x,y=meq.y,z=meq.z,mode='markers'
                                ,marker=dict(size=3,color=meq.datenums,colorscale='RdBu',
                                        ))])
fig.add_scatter3d(x=swells[swells["HoleID"]=='E1-I'].x, # type: ignore
                    y=swells[swells["HoleID"]=='E1-I'].y, # type: ignore
                    z=swells[swells["HoleID"]=='E1-I'].z, # type: ignore
                    mode='lines',
marker=dict(
        size=3,
        color='black',
        opacity=0.8))
fig.add_scatter3d(x=swells[swells["HoleID"]=='E1-P'].x, # type: ignore
                    y=swells[swells["HoleID"]=='E1-P'].y, # type: ignore
                    z=swells[swells["HoleID"]=='E1-P'].z, # type: ignore
                    mode='lines',
marker=dict(
        size=3,
        color='black',
        opacity=0.8))
fig.add_scatter3d(x=pdtpts[:,0], y=pdtpts[:,1],z=pdtpts[:,2],mode='lines',
marker=dict(
        size=3,
        color='red',
        opacity=0.8))
fig.add_scatter3d(x=pdbpts[:,0], y=pdbpts[:,1],z=pdbpts[:,2],mode='lines',
marker=dict(
        size=3,
        color='red',
        opacity=0.8))
fig.add_scatter3d(x=pstpts[:,0], y=pstpts[:,1],z=pstpts[:,2],mode='lines',
marker=dict(
        size=3,
        color='blue',
        opacity=0.8))
fig.add_scatter3d(x=psbpts[:,0], y=psbpts[:,1],z=psbpts[:,2],mode='lines',
marker=dict(
        size=3,
        color='blue',
        opacity=0.8))
fig.add_scatter3d(x=obpts[:,0], y=obpts[:,1],z=obpts[:,2],mode='lines',
marker=dict(
        size=3,
        color='green',
        opacity=0.8))
fig.add_scatter3d(x=otpts[:,0], y=otpts[:,1],z=otpts[:,2],mode='lines',
marker=dict(
        size=3,
        color='green',
        opacity=0.8))

# for k in range(nplanes):
#         # normV = Normalize(vmin=0,vmax=10000)
#         fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
#         i=allsurf_tri_arr[k,:,0],j=allsurf_tri_arr[k,:,1],k=allsurf_tri_arr[k,:,2],
#         colorscale='thermal',facecolor=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
#         intensity=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
#         intensitymode='cell',
#         cmin=np.min(np.array([mest[mfrac[k][0][0]:mfrac[k][0][1]]] + [mest[mfrac[k][1][0]:mfrac[k][1][1]]] + [mest[mfrac[k][2][0]:mfrac[k][2][1]]])),
#         cmax=np.max(np.array([mest[mfrac[k][0][0]:mfrac[k][0][1]]] + [mest[mfrac[k][1][0]:mfrac[k][1][1]]] + [mest[mfrac[k][2][0]:mfrac[k][2][1]]]))))
# for k in range(nplanes):
#         # normV = Normalize(vmin=0,vmax=10000)
#         fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
#         i=allsurf_tri_arr[k,:,0],j=allsurf_tri_arr[k,:,1],k=allsurf_tri_arr[k,:,2],
#         colorscale='thermal',facecolor=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
#         intensity=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
#         intensitymode='cell',
#         cmin=min(mest[mode[i][0]:mode[i][1]]),
#         cmax=max(mest[mode[i][0]:mode[i][1]])))
        # cmin=-np.percentile(mest[mode[i][0]:mode[i][1]],99),
        # cmax=np.percentile(mest[mode[i][0]:mode[i][1]],99)))
for k in range(nplanes):
    # normV = Normalize(vmin=0,vmax=10000)
    fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
                i=np.array(allsurf_tris[k])[:,0],j=np.array(allsurf_tris[k])[:,1],k=np.array(allsurf_tris[k])[:,2],
                colorscale='thermal',facecolor=allslip[k],intensity=allslip[k],intensitymode='cell',
                cmin=-np.min(np.percentile(np.array(allslip),99)),
                cmax=np.max(np.percentile(np.array(allslip),99))))
# for k in range(nplanes):
#     # normV = Normalize(vmin=0,vmax=10000)
#     fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
#                 i=allsurf_tri_arr[k,:,0],j=allsurf_tri_arr[k,:,1],k=allsurf_tri_arr[k,:,2],
#                 colorscale='YlOrRd',facecolor=covslip[k],intensity=covslip[k],intensitymode='cell',cmin=1000,cmax=100000))
# for k in range(nplanes):
#     # normV = Normalize(vmin=0,vmax=10000)
#     fig.add_trace(go.Mesh3d(x=allsurf_pts[k][:,0], y=allsurf_pts[k][:,1], z=allsurf_pts[k][:,2],
#                 i=allsurf_tri_arr[k,:,0],j=allsurf_tri_arr[k,:,1],k=allsurf_tri_arr[k,:,2],
#                 colorscale='YlOrRd',facecolor=np.array(modcov[k]),intensity=np.array(modcov[k]),intensitymode='cell',cauto=True))
fig.update_layout(
        autosize=False,
        width=1200,
        height=800,)
fig.show()

In [ ]:
for j in range(len(mest)):
    nplanes = 6
    fracList = [f'OTP','West North','West South','West Deep','East North','East South']
    titles=[f'Strike Slip ','Dip Slip','Tensile ']
    fig,ax = plt.subplots(nplanes,3,figsize=(12,10))
    mode = genSlice(len(mest[j]),3)
    ifrac = fracSlice(mode[0][1],nplanes)
    pfrac = fracSlice(len(allsurf_pts),nplanes)
    for i in range(3):
        for k in range(nplanes):
        # mode = list(sliced((0,8,8,16,16,24),2))

            # vm = np.percentile(mest[j][mode[i][0]:mode[i][1]],99)
            vm = np.percentile(mest[j][mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],99)


            # im  = ax[k,i].tripcolor(allsurf_pts[pfrac[k][0]:pfrac[k][1], 0], allsurf_pts[pfrac[k][0]:pfrac[k][1], 2],
            #                         allsurf_tris[ifrac[k][0]:ifrac[k][1]],
            #                         facecolors=mest[mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
            #                         cmap='seismic',vmin=-50000,vmax=50000) # type: ignore
            
            im  = ax[k,i].tripcolor(np.array(allsurf_pts[k])[:,0], np.array(allsurf_pts[k])[:,2],
                            np.array(allsurf_tris[k]),
                            facecolors=mest[j][mode[i][0]:mode[i][1]][ifrac[k][0]:ifrac[k][1]],
                            cmap='seismic',vmin=-vm,vmax=vm) # type: ignore
            ax[k,i].set_xlabel('X direction (m)')
            ax[k,i].set_ylabel('Z direction (m)')
            # ax[i].axis('equal')
            # ax[i].set_aspect('equal','box')
            plt.colorbar(im,ax=ax[k,i])
            ax[k,i].set_title(f'{fracList[k]} {titles[i]}')
            plt.suptitle(f'{DASdates[timeEpochs[j]]}')
            fig.tight_layout()

In [ ]:

def lineSearch(x, g, p, func, t=0.5):
	"""
	Line Search Function
	simple descent line search

	input
	-----
	x : array_like
		Base point.
	g : array_like
		Gradient on the base point.
	p : array_like
		Given descent direction.
	func : function
		Input x and return funciton value.
	t : float, optional
		step size shrink ratio

	output
	------
	step_size : float or None
		When sucess return the step_size, otherwise return None.
	"""
	# initial step size
	step_size = 1.0
	# 
	m = g.dot(p)
	if m < 0:
		print('Line search: not a descent direction.')
		return None
	#
	f = func(x)
	y = x - step_size*p
	#
	while func(y) > f:
		step_size *= t
		if step_size < 1e-15:
			print('Line search: step size too small.')
			return None
		y = x - step_size*p
	#
	return step_size

# line search function - armijo line search
# -----------------------------------------------------------------------------
def lineSearch_armijo(x, g, p, func, c=0.01, t=0.5):
	"""
	Line Search Function
	armijo line search

	input
	-----
	x : array_like
		Base point.
	g : array_like
		Gradient on the base point.
	p : array_like
		Given descent direction.
	func : function
		Input x and return funciton value.
	c : float, optional
		has to strictly be between 0 and 1
	t : float, optional
		step size shrink ratio

	output
	------
	step_size : float or None
		When sucess return the step_size, otherwise return None.
	"""
	# save guard for c
	assert 0 < c < 1, 'c needs to strictly be in 0 and 1'
	# initial step size
	step_size = 1.0
	# 
	m = c*g.dot(p)
	if m < 0:
		print('Line search: not a descent direction.')
		return None
	#
	f = func(x)
	y = x - step_size*p
	#
	while func(y) > f - step_size*m:
		step_size *= t
		if step_size < 1e-15:
			print('Line search: step size too small.')
			return None
		y = x - step_size*p
	#
	return step_size


def optimizeWithGD(x0, func, grad, step_size, tol=1e-7, max_iter=10000, use_line_search=False):
    """
    Optimize with Gradient Descent
    
    input
    -----
    x0 : array_like
        Starting point for the solver.
    func : function
        Takes x and return the obj function value.
    grad : function
        Takes x and returns the gradient of "func".
    step_size : float or None
        If it is a float number and `use_line_search=False`, it will be used as the step size.
        Otherwise, line search will be used
    tol : float, optional
        Gradient tolerance for terminating the solver.
    max_iter : int, optional
        Maximum number of iterations for terminating the solver.
    use_line_search : bool, optional
        When it is true a line search will be used, other wise `step_size` has to be provided.
        
    output
    ------
    x : array_like
        Final solution
    obj_his : array_like
        Objective function's values convergence history
    err_his : array_like
        Norm of gradient convergence history
    exit_flag : int
        0, norm of gradient below `tol`
        1, exceed maximum number of iteration
        2, line search fail
        3, other
    """
    # safeguard
    if not use_line_search and step_size is None:
        print('Please specify the step_size or use the line search.')
        return x0, np.array([]), np.array([]), 3
    
    # initial step
    x = np.copy(x0)
    g = grad(x)
    #
    obj = func(x)
    err = norm(g)
    #
    obj_his = np.zeros(max_iter + 1)
    err_his = np.zeros(max_iter + 1)
    #
    obj_his[0] = obj
    err_his[0] = err
    
    # start iterations
    iter_count = 0
    while err >= tol:
        if use_line_search:
            step_size = lineSearch(x, g, g, func)
        #
        # if line search fail step_size will be None
        if step_size is None:
            print('Gradient descent line search fail.')
            return x, obj_his[:iter_count+1], err_his[:iter_count+1], 2
        #
        # gradient descent step
        x -= step_size * g
        # update obj function and gradient and error
        g = grad(x)
        #
        obj = func(x)
        err = norm(g)
        #
        iter_count += 1
        obj_his[iter_count] = obj
        err_his[iter_count] = err
        #
        # check if exceed maximum number of iteration
        if iter_count >= max_iter:
            print('Gradient descent reached maximum number of iteration.')
            return x, obj_his[:iter_count+1], err_his[:iter_count+1], 1
    #
    return x, obj_his[:iter_count+1], err_his[:iter_count+1], 0

def optimizeWithPGD(x0, func_f, func_g, grad_f, prox_g, beta_f, tol=1e-5, max_iter=50000):
    """
    Optimize with Proximal Gradient Descent Method
        min_x f(x) + g(x)
    where f is beta smooth and g is proxiable.
    
    input
    -----
    x0 : array_like
        Starting point for the solver
    func_f : function
        Input x and return the function value of f
    func_g : function
        Input x and return the function value of g
    grad_f : function
        Input x and return the gradient of f
    prox_g : function
        Input x and a constant float number and return the prox solution
    beta_f : float
        beta smoothness constant for f
    tol : float, optional
        Gradient tolerance for terminating the solver.
    max_iter : int, optional
        Maximum number of iteration for terminating the solver.
        
    output
    ------
    x : array_like
        Final solution
    obj_his : array_like
        Objective function value convergence history
    err_his : array_like
        Norm of gradient convergence history
    exit_flag : int
        0, norm of gradient below `tol`
        1, exceed maximum number of iteration
        2, others
    """
    # initial information
    x = x0.copy()
    g = grad_f(x)
    #
    step_size = 1.0/beta_f
    # not recording the initial point since we do not have measure of the optimality
    obj_his = np.zeros(max_iter)
    err_his = np.zeros(max_iter)
    
    # start iteration
    iter_count = 0
    err = tol + 1.0
    while err >= tol:
        #####
        # the proximal gradient step
        x_new = prox_g(x - step_size*g,step_size)
        #####
        # update information
        obj = func_f(x_new) + func_g(x_new)
        err = norm(x - x_new)/step_size
        #
        np.copyto(x, x_new)
        g = grad_f(x)
        #
        obj_his[iter_count] = obj
        err_his[iter_count] = err
        #
        # check if exceed maximum number of iteration
        iter_count += 1
        if iter_count >= max_iter:
            print('Proximal gradient descent reach maximum of iteration')
            return x, obj_his[:iter_count], err_his[:iter_count], 1
    #
    return x, obj_his[:iter_count], err_his[:iter_count], 0
# Newton's Method
# -----------------------------------------------------------------------------
def optimizeWithNT(x0, func, grad, hess, tol=1e-6, max_iter=100):
    """
    Optimize with Newton's Method
    
    input
    -----
    x0 : array_like
        Starting point for the solver.
    func : function
        Input x and return the function value.
    grad : function
        Input x and return the gradient.
    hess : function
        Input x and return the Hessian matrix.
    tol : float, optional
        Gradient tolerance for terminating the solver.
    max_iter : int, optional
        Maximum number of iteration for terminating the solver.
        
    output
    ------
    x : array_like
        Final solution
    obj_his : array_like
        Objective function value convergence history
    err_his : array_like
        Norm of gradient convergence history
    exit_flag : int
        0, norm of gradient below `tol`
        1, exceed maximum number of iteration
        2, others
    """
    # initial step
    x = np.copy(x0)
    g = grad(x)
    H = hess(x)
    #
    obj = func(x)
    err = norm(g)
    #
    obj_his = np.zeros(max_iter + 1)
    err_his = np.zeros(max_iter + 1)
    #
    obj_his[0] = obj
    err_his[0] = err
    
    # start iteration
    iter_count = 0
    while err >= tol:
        # Newton's step
        x -= solve(H, g)
        #
        # update function, gradient and Hessian
        g = grad(x)
        H = hess(x)
        #
        obj = func(x)
        err = norm(g)
        #
        iter_count += 1
        obj_his[iter_count] = obj
        err_his[iter_count] = err
        #
        # check if exceed maximum number of iteration
        if iter_count >= max_iter:
            print('Gradient descent reach maximum number of iteration.')
            return x, obj_his[:iter_count+1], err_his[:iter_count+1], 1
    #
    return x, obj_his[:iter_count+1], err_his[:iter_count+1], 0

In [ ]:
# define the function, prox and the beta constant
def func_f_cs(x):
    f_cs = (0.5)*norm(A@x - b,2)**2
    return f_cs
def func_g_cs(x):
    g_cs = lam_cs*norm(x,2)
    return g_cs
def grad_f_cs(x):
    Ax = A@x
    grad_f = A.T@(Ax - b) 
    return grad_f
def prox_g_cs(x, t):
    # prox of 1 norm
    prox_gx = np.sign(x)*np.maximum(np.abs(x) - lam_cs*t, 0)
    # prox of 2 norm
    # prox_gx = np.maximum(0,(1 - (t / norm(x,2)))) * x
    # if norm(x,2) <= t:
    #     prox_gx = 0
    # elif norm(x,2) > t:
    #     prox_gx = ((norm(x,2) - t) / norm(x,2)) * x 
        
    return prox_gx
def prox_g_ps(x,t):
    # prox of 2 norm
    prox_gx = np.maximum(0,(1 - (t / norm(x,2)))) * x
    return prox_gx
beta_f_cs = norm(A.T, 2)**2
